In [14]:
import numpy as np
import pandas as pd


def calculate_mean(time_points: pd.Series, values: pd.Series) -> float:
    prev_time_point = time_points.iloc[0]
    delay_sum = 0.0
    value_sum = 0.0

    for time_point, value in zip(time_points.iloc[1:], values):
        delay = time_point - prev_time_point
        prev_time_point = time_point
        delay_sum += delay
        value_sum += value * delay

    try:
        res = value_sum / delay_sum
        return res
    except ZeroDivisionError:
        return 0

def calculate_std_dev(time_points: pd.Series, values: pd.Series, mean: float) -> float:
    prev_time_point = time_points.iloc[0]
    delay_sum = 0.0
    value_sum = 0.0

    for time_point, value in zip(time_points.iloc[1:], values):
        delay = time_point - prev_time_point
        prev_time_point = time_point
        delay_sum += delay
        value_sum += ((value - mean) ** 2) * delay

    try:
        res = np.sqrt(value_sum / delay_sum)
        return res
    except ZeroDivisionError:
        return 0

In [15]:
import os
from pathlib import Path
import pandas as pd
import attr
from typing import Optional

verification_data_dir = Path('./verification_data')

@attr.frozen
class ParamsData:
    common_props: pd.DataFrame
    time_wait_allocate: pd.DataFrame
    time_in_system: pd.DataFrame

datas: dict[tuple[int, ...], ParamsData] = {}

for dirpath, dir, filenames in os.walk(verification_data_dir):
    dir_path = Path(dirpath)
    if dir_path.name == verification_data_dir.name:
        continue
    params = tuple(int(n) for n in dir_path.name.split('_'))

    common_props: Optional[pd.DataFrame] = None
    time_wait_allocate: Optional[pd.DataFrame] = None
    time_in_system: Optional[pd.DataFrame] = None
    for file_name in filenames:
        data = pd.read_csv(Path(dirpath) / file_name)
        if file_name.startswith('commonProps'):
            common_props = data
        elif file_name.startswith('timeWaitAllocate'):
            time_wait_allocate = data
        elif file_name.startswith('timeInSystem'):
            time_in_system = data
    
    if common_props is not None and time_wait_allocate is not None and time_in_system is not None:
        datas[params] = ParamsData(common_props, time_wait_allocate, time_in_system)
    else:
        raise Exception('empty data')
    

In [16]:
from array import array

@attr.frozen
class MeanStddevStats:
    diskLoad_mean: array[float] = attr.field(init=False, factory=lambda: array('d'))
    diskLoad_std_dev: array[float] = attr.field(init=False, factory=lambda: array('d'))
    ioChannelLoad_mean: array[float] = attr.field(init=False, factory=lambda: array('d'))
    ioChannelLoad_std_dev: array[float] = attr.field(init=False, factory=lambda: array('d'))
    processorsLoad_mean: array[float] = attr.field(init=False, factory=lambda: array('d'))
    processorsLoad_std_dev: array[float] = attr.field(init=False, factory=lambda: array('d'))
    totalWaitAllocate_mean: array[float] = attr.field(init=False, factory=lambda: array('d'))
    totalWaitAllocate_std_dev: array[float] = attr.field(init=False, factory=lambda: array('d'))
    useOfPage_mean: array[float] = attr.field(init=False, factory=lambda: array('d'))
    useOfPage_std_dev: array[float] = attr.field(init=False, factory=lambda: array('d'))
    timeInSystem_mean: array[float] = attr.field(init=False, factory=lambda: array('d'))
    timeInSystem_std_dev: array[float] = attr.field(init=False, factory=lambda: array('d'))
    timeWaitAllocate_mean: array[float] = attr.field(init=False, factory=lambda: array('d'))
    timeWaitAllocate_std_dev: array[float] = attr.field(init=False, factory=lambda: array('d'))

mean_stddev_stats_list: list[tuple[tuple[int, ...], pd.DataFrame]] = []

for params, params_data in datas.items():
    mean_stddev_stats = MeanStddevStats()

    for run_num, group in params_data.common_props.groupby('runNumber'):
        # Calculate means and standard deviations
        diskLoad_mean = calculate_mean(group['timePoint'], group['diskLoad'])
        diskLoad_std_dev = calculate_std_dev(group['timePoint'], group['diskLoad'], diskLoad_mean)

        ioChannelLoad_mean = calculate_mean(group['timePoint'], group['ioChannelLoad'])
        ioChannelLoad_std_dev = calculate_std_dev(group['timePoint'], group['ioChannelLoad'], ioChannelLoad_mean)

        processorsLoad_mean = calculate_mean(group['timePoint'], group['processorsLoad'])
        processorsLoad_std_dev = calculate_std_dev(group['timePoint'], group['processorsLoad'], processorsLoad_mean)

        totalWaitAllocate_mean = calculate_mean(group['timePoint'], group['totalWaitAllocate'])
        totalWaitAllocate_std_dev = calculate_std_dev(group['timePoint'], group['totalWaitAllocate'], totalWaitAllocate_mean)

        useOfPage_mean = calculate_mean(group['timePoint'], group['useOfPage'])
        useOfPage_std_dev = calculate_std_dev(group['timePoint'], group['useOfPage'], useOfPage_mean)

        mean_stddev_stats.diskLoad_mean.append(diskLoad_mean)
        mean_stddev_stats.diskLoad_std_dev.append(diskLoad_std_dev)

        mean_stddev_stats.ioChannelLoad_mean.append(ioChannelLoad_mean)
        mean_stddev_stats.ioChannelLoad_std_dev.append(ioChannelLoad_std_dev)

        mean_stddev_stats.processorsLoad_mean.append(processorsLoad_mean)
        mean_stddev_stats.processorsLoad_std_dev.append(processorsLoad_std_dev)

        mean_stddev_stats.totalWaitAllocate_mean.append(totalWaitAllocate_mean)
        mean_stddev_stats.totalWaitAllocate_std_dev.append(totalWaitAllocate_std_dev)

        mean_stddev_stats.useOfPage_mean.append(useOfPage_mean)
        mean_stddev_stats.useOfPage_std_dev.append(useOfPage_std_dev)
    
    for run_num, group in params_data.time_in_system.groupby('runNumber'):
        timeInSystem_mean = calculate_mean(group['timePoint'], group['timeInSystem'])
        timeInSystem_std_dev = calculate_std_dev(group['timePoint'], group['timeInSystem'], timeInSystem_mean)
        mean_stddev_stats.timeInSystem_mean.append(timeInSystem_mean)
        mean_stddev_stats.timeInSystem_std_dev.append(timeInSystem_std_dev)

    for run_num, group in params_data.time_wait_allocate.groupby('runNumber'):
        timeWaitAllocate_mean = calculate_mean(group['timePoint'], group['timeWaitAllocate'])
        timeWaitAllocate_std_dev = calculate_std_dev(group['timePoint'], group['timeWaitAllocate'], timeWaitAllocate_mean)
        mean_stddev_stats.timeWaitAllocate_mean.append(timeWaitAllocate_mean)
        mean_stddev_stats.timeWaitAllocate_std_dev.append(timeWaitAllocate_std_dev)

    mean_stddev_stats_list.append((params, pd.DataFrame(attr.asdict(mean_stddev_stats))))

In [42]:
params_mat: list[pd.Series] = []
global_mean_stddev_list: list[pd.DataFrame] = []
mean_stddev_stats_relative_mean_list: list[pd.DataFrame] = []

for i, params_mean_stddev_stats in enumerate(mean_stddev_stats_list):
    params = params_mean_stddev_stats[0]
    mean_stddev_stats = params_mean_stddev_stats[1]
    params_mat.append(pd.Series(dict(
        pagesNum=params[0],
        processorsNum=params[1],
        diskNum=params[2],
        pagesStart=params[3],
        pagesEnd=params[4],
        tasksTimeMean=params[5]
    )))
    means = mean_stddev_stats.mean()
    global_mean_stddev_list.append(means)
    mean_stddev_stats_relative_mean = (mean_stddev_stats - means).abs() * 100 / means
    mean_stddev_stats_relative_mean.fillna(0, inplace=True)
    mean_stddev_stats_relative_mean['params_index'] = i
    mean_stddev_stats_relative_mean_list.append(mean_stddev_stats_relative_mean)

In [43]:
params_data_frame = pd.concat(params_mat, axis=1)
params_data_frame = params_data_frame.T
params_data_frame

,pagesNum,processorsNum,diskNum,pagesStart,pagesEnd,tasksTimeMean
0,1200,40,50,100,200,14
1,250,4,5,30,40,8
2,1200,50,50,10,200,4
3,1000,40,8,100,200,9
4,200,4,4,20,60,7
5,400,10,10,70,100,8
6,700,12,12,100,150,8
7,1000,30,30,70,170,15
8,131,2,4,20,60,5


In [44]:
global_mean_stddev_data_frame = pd.concat(global_mean_stddev_list, axis=1)
global_mean_stddev_data_frame = global_mean_stddev_data_frame.T
global_mean_stddev_data_frame

,diskLoad_mean,diskLoad_std_dev,ioChannelLoad_mean,ioChannelLoad_std_dev,processorsLoad_mean,processorsLoad_std_dev,totalWaitAllocate_mean,totalWaitAllocate_std_dev,useOfPage_mean,useOfPage_std_dev,timeInSystem_mean,timeInSystem_std_dev,timeWaitAllocate_mean,timeWaitAllocate_std_dev
0,0.002690,0.000057,0.001072,0.000024,0.712881,0.010671,0.000000,0.000000,106.978137,82.259688,9.471274,2.942525,0.000000,0.000000
1,0.004698,0.000095,0.001868,0.000034,1.248875,0.019565,0.000000,0.000000,43.239251,23.067334,9.485341,2.893644,0.000000,0.000000
2,0.006213,0.000123,0.002485,0.000053,1.669911,0.044725,4155.356341,2393.336541,1158.592940,36.082313,393.825201,2416.826722,487.930083,2815.079645
3,0.004171,0.000074,0.001669,0.000027,1.108247,0.014637,0.000000,0.000000,165.794027,97.635997,9.454172,2.908708,0.000000,0.000000
4,0.005344,0.000089,0.002140,0.000039,1.425913,0.017412,0.001409,0.030715,56.513423,31.442723,9.574777,3.191714,1.648168,1.434776
5,0.004660,0.000087,0.001877,0.000035,1.253484,0.016017,0.000034,0.004977,106.007599,56.736385,9.503510,2.905869,1.202245,0.000000
6,0.004680,0.000080,0.001863,0.000035,1.246650,0.015633,0.000020,0.001989,155.304455,83.607824,9.503564,2.953817,0.297126,0.590091
7,0.002512,0.000049,0.001000,0.000021,0.669379,0.011898,0.000000,0.000000,79.964713,66.505623,9.521851,2.951505,0.000000,0.000000
8,0.006208,0.000113,0.002490,0.000040,1.656594,0.020292,1706.802430,965.390204,112.911580,12.542579,437.758691,3511.420928,586.624594,4138.392153


In [47]:
mean_stddev_stats_relative_mean_data_frame = pd.concat(mean_stddev_stats_relative_mean_list, ignore_index=True)
mean_stddev_stats_relative_mean_data_frame

,diskLoad_mean,diskLoad_std_dev,ioChannelLoad_mean,ioChannelLoad_std_dev,processorsLoad_mean,processorsLoad_std_dev,totalWaitAllocate_mean,totalWaitAllocate_std_dev,useOfPage_mean,useOfPage_std_dev,timeInSystem_mean,timeInSystem_std_dev,timeWaitAllocate_mean,timeWaitAllocate_std_dev,params_index
0,0.101265,5.104997,0.326192,10.464096,0.212164,2.002064,0.000000,0.000000,0.648024,0.142229,0.087886,0.204529,0.000000,0.000000,0
1,1.118247,5.783631,0.170958,2.950048,0.375131,2.983298,0.000000,0.000000,0.783768,0.171292,0.338954,0.096485,0.000000,0.000000,0
2,1.148900,0.029906,0.491483,2.698446,0.137783,20.952253,0.000000,0.000000,0.284628,0.025916,0.059248,1.376571,0.000000,0.000000,0
3,0.219326,8.874560,0.463395,2.114684,0.517918,23.616694,0.000000,0.000000,0.810926,0.081692,0.725667,0.052988,0.000000,0.000000,0
4,0.087408,2.043974,0.127146,6.930286,0.217167,7.649802,0.000000,0.000000,0.390555,0.084839,0.358075,1.128546,0.000000,0.000000,0
5,0.301966,18.584058,0.096498,0.044371,0.310272,37.593161,0.000000,0.000000,0.330425,0.009924,0.457286,1.052444,0.000000,0.000000,1
6,1.375531,25.821791,0.523249,7.061189,0.534949,3.865994,0.000000,0.000000,0.252837,0.174710,0.452133,1.494867,0.000000,0.000000,1
7,0.239186,21.220559,0.092869,1.744201,0.067437,13.965617,0.000000,0.000000,0.596371,0.008139,0.015487,0.640135,0.000000,0.000000,1
8,0.715054,6.376210,1.237360,5.397404,0.996991,17.962828,0.000000,0.000000,0.931886,0.160517,0.193215,0.059748,0.000000,0.000000,1
9,0.119324,20.359037,0.524744,10.670021,0.839751,1.798723,0.000000,0.000000,0.413103,0.012409,0.213855,0.137964,0.000000,0.000000,1
